In [1]:
import torch
import Models.misc as misc
import Inference.Kolmogorov.performance as performance
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Emu_file_path = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250526_223850-r12kgbg1/files/checkpoint_best.p'
CT_file_path = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250715_210115-naiteaau/files/checkpoint_last.p'

CT = misc.load_diffusion_model(CT_file_path).to(device)
Emu = misc.load_model(Emu_file_path).to(device)

In [2]:
data_dict = torch.load("/scratch/ql2221/thermalizer_data/kolmogorov/reynold10k/test_long.p")

In [3]:
data = data_dict["data"]
print(data.shape)
x = data[0,:,:,:]
x = x.unsqueeze(0).to(device)
print(x.shape)
x = x/4.44

torch.Size([5, 70000, 64, 64])
torch.Size([1, 70000, 64, 64])


In [4]:
delta = torch.tensor([1]).to(device)
rollout, _ = performance.run_conditional_emu(x[:,0:1], Emu, CT, n_steps=1000, delta = delta, denoising_steps=5, freq = 25, silent=True, sigma=None, Regression = True)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!